In [1]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

# Creating Tools
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=1000)
arxiv =  ArxivQueryRun(api_wrapper=api_wrapper_arxiv, description="Search for academic papers on arXiv.org")

api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=600)
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia, description="Search for articles on Wikipedia.org")

print(arxiv.invoke("Quantum Computing"))
print(wikipedia.invoke("Quantum Computing"))

Published: 2022-08-01
Title: The Rise of Quantum Internet Computing
Authors: Seng W. Loke
Summary: This article highlights quantum Internet computing as referring to
distributed quantum computing over the quantum Internet, analogous to
(classical) Internet computing involving (classical) distributed computing over
the (classical) Internet. Relevant to quantum Internet computing would be areas
of study such as quantum protocols for distributed nodes using quantum
information for computations, quantum cloud computing, delegated verifiable
blind or private computing, non-local gates, and distributed quantum
applications, over Internet-scale distances.

Published: 2000-03-31
Title: Unconventional Quantum Computing Devices
Authors: Seth Lloyd
Summary: This paper investigates a variety of unconventional quantum computation
devices, including fermionic quantum computers and computers that exploit
nonlinear quantum mechanics. It is shown that unconventional quantum computing
devices can in pri

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
load_dotenv()
import os

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

tavily = TavilySearchResults()

print(tavily.invoke("provide me the recent AI news"))

[{'title': 'Artificial Intelligence News - ScienceDaily', 'url': 'https://www.sciencedaily.com/news/computers_math/artificial_intelligence/', 'content': '![ScienceDaily](/images/sd-logo.png)\n\n# Artificial Intelligence News\n\n## Top Headlines\n\n## Latest Headlines\n\n## Earlier Headlines\n\n### Wednesday, May 7, 2025\n\n### Tuesday, May 6, 2025\n\n### Monday, May 5, 2025\n\n### Thursday, May 1, 2025\n\n### Wednesday, April 30, 2025\n\n### Friday, April 25, 2025\n\n### Thursday, April 24, 2025\n\n### Wednesday, April 23, 2025\n\n### Tuesday, April 22, 2025\n\n### Tuesday, April 15, 2025\n\n### Monday, April 14, 2025\n\n### Wednesday, April 9, 2025\n\n### Tuesday, April 8, 2025 [...] ### Wednesday, February 19, 2025\n\n### Thursday, February 13, 2025\n\n### Monday, February 10, 2025\n\n### Monday, February 3, 2025\n\n### Wednesday, January 29, 2025\n\n### Tuesday, January 28, 2025\n\n### Monday, January 27, 2025\n\n### Tuesday, January 21, 2025\n\n### Thursday, January 16, 2025\n\n###

In [5]:
tools = [arxiv, wikipedia, tavily]

In [7]:
from langchain_groq import ChatGroq

llm = ChatGroq(model = "qwen-qwq-32b")

In [8]:
llm.invoke("latest llm model of qwen")

AIMessage(content="\n<think>\nOkay, the user is asking about the latest LLM model from Qwen. Let me start by recalling what I know about Qwen's models. I remember that Qwen has several versions, like Qwen1, Qwen1.5, Qwen2, and so on. The latest one I know of is Qwen2.5. I should confirm that.\n\nWait, but maybe there's an even newer version now? Sometimes companies release updates without much fanfare. Let me think. The user might be looking for the latest available as of the current date. Since I can't check real-time data, I'll have to go with what's documented up to my last update in December 2024.\n\nSo, Qwen2.5 is the most recent version mentioned in the official documentation. It's an improvement over previous models with better performance, efficiency, and larger context lengths. I should mention the key features of Qwen2.5, like the context window, training data, and capabilities. Also, it's important to highlight that different versions cater to different use cases, so users m

In [9]:
llm_with_tools = llm.bind_tools(tools=tools)

In [10]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages 

In [11]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage] , add_messages] # add_messages will append all the messages to the messages varaible and return in the typed dict format, it will not over write 
    

In [12]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START , END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [ ]:
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))